In [144]:
#Mocap Arena Camera Calibrate Program
#Mateus Pincho - calibrate intrisics
import argparse
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [145]:
import plotly.graph_objects as go

class Feed: 
    def __init__(self, title, res=(480,480), graphical=False):
        self.title = title
        self.graphical = graphical # Toggle to activate graphical mode
        self.res = res # Change feed dimensions 

        # Create Figure 
        self.figure = go.Figure(
            layout=go.Layout(
                height=700, 
                width=900, 
                title=go.layout.Title(text=self.title)
            )
        )

        self.figure.update_yaxes(
            scaleanchor="x",
            scaleratio=1
        )
        
        self.figure.update_layout(
            xaxis_title='x',
            yaxis_title='y',
            plot_bgcolor='white',
            font=dict(
                family='Arial',
                size=15,
                color='black'
            ),
            xaxis=dict(
                gridcolor='lightgray',
                dtick = res[0]/10,
                range=[0, self.res[0]]
            ),
            yaxis=dict(
                gridcolor='lightgray',
                dtick = res[1]/10,
                range=[self.res[1], 0]
            )
        )

        self.figure.add_shape(
            type='rect',
            x0=0, y0=0, x1=res[0], y1=res[1],
            line=dict(color='black'),
        )


    def add_points(self, point, name, color=None):
        self.figure.add_trace(
            go.Scatter(
                x=point[0],
                y=point[1],
                mode='markers',
                marker=dict(
                    size=5,
                    opacity=0.80,
                    color=color
                ),
                name=name,
                legendgroup='Points',
                legendgrouptitle_text='Points',
                showlegend=self.graphical
            )
        )

#Reconstrução do tabuleiro
def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize # Square size não interfere na calibração -> somente um fator de escala
                       # o square size não é utilizado na calibração dos cara de stanford
    return X

#Detectando os corners
#Detectando os corners
def detectCorners(images, boardPoints, patternSize):
    worldPoints = []
    imagePoints = [] 

    img_size = 0
    counter = 0
    for fname in images:
        img = cv2.imread(fname)
        print(img_size)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_size = gray.shape[::-1]
        ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
        if ret == True:
            print("Corners found in image " + str(fname)) #- see if corners are found 
            #corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
            worldPoints.append(boardPoints)
            imagePoints.append(corners)
            counter+=1

    print("Corners found in " + str(counter) + " images")
    
    return worldPoints, imagePoints, img_size


def check_reprojection_error(real_image_points, world_points, rvec, tvec, camera_matrix, dist_coeff):

    projected_points, _ = cv2.projectPoints(world_points, rvec, tvec, camera_matrix, dist_coeff)

    perViewError = cv2.norm(real_image_points, projected_points, cv2.NORM_L2) / len(real_image_points)

    return perViewError

#Descobrindo os intrisicos, extrinsecos e o erro de reprojeção 
def calibrate(images, useFisheye, patternSize, squareSize, imgSize):

    boardPoints = construct3DPoints(patternSize, squareSize)

    worldPoints, imagePoints, _ = detectCorners(images, boardPoints, patternSize)

    if useFisheye:
        flagsCalib = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW+cv2.fisheye.CALIB_CHECK_COND
        calibrateCriteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER,30, 1e-12)

        ret, cameraMatrix, k, R, t = cv2.fisheye.calibrate(np.expand_dims(np.asarray(worldPoints), -2), imagePoints, imgSize, None, None,
                                                                    flags=flagsCalib,criteria=calibrateCriteria)
    else:
        flagsCalib = cv2.CALIB_RATIONAL_MODEL

        ret, cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = cv2.calibrateCameraExtended(worldPoints, imagePoints, imgSize, None, None,
                                                                flags=flagsCalib)

    print("RMS re-projection error:", ret)
    print("The median re-projection error", np.median(perViewError))
    print("Camera Matrix:\n", cameraMatrix)
    print("Distortion Parameters:\n", k)

    return cameraMatrix, k, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError

In [146]:
useFisheye = False
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#images = glob.glob('../../images/chess/virtual/*.jpg')

images_set = ['../../images/virtual/set_5/*.jpg',
              '../../images/virtual/set_10/*.jpg',
              '../../images/virtual/set_15/*.jpg',
              '../../images/virtual/set_20/*.jpg',
              '../../images/virtual/set_25/*.jpg']


images = glob.glob('../../images/virtual/set_25/*.jpg')
patternSize = (7,7)
squareSize = 30
imgSize = (480,480)

print(images)

['../../images/virtual/set_25/image10.jpg', '../../images/virtual/set_25/image4.jpg', '../../images/virtual/set_25/image20.jpg', '../../images/virtual/set_25/image14.jpg', '../../images/virtual/set_25/image8.jpg', '../../images/virtual/set_25/image1.jpg', '../../images/virtual/set_25/image15.jpg', '../../images/virtual/set_25/image22.jpg', '../../images/virtual/set_25/image9.jpg', '../../images/virtual/set_25/image13.jpg', '../../images/virtual/set_25/image17.jpg', '../../images/virtual/set_25/image6.jpg', '../../images/virtual/set_25/image7.jpg', '../../images/virtual/set_25/image18.jpg', '../../images/virtual/set_25/image19.jpg', '../../images/virtual/set_25/image12.jpg', '../../images/virtual/set_25/image21.jpg', '../../images/virtual/set_25/image24.jpg', '../../images/virtual/set_25/image2.jpg', '../../images/virtual/set_25/image23.jpg', '../../images/virtual/set_25/image25.jpg', '../../images/virtual/set_25/image3.jpg', '../../images/virtual/set_25/image16.jpg', '../../images/virt

In [147]:
for idx, set  in enumerate(images_set):
    print('\n Using set ' + str((idx + 1)*5) )
    images = glob.glob(set)

    cameraMatrix, distCoeff, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = calibrate(images, useFisheye, patternSize,squareSize, imgSize) 


 Using set 5
0
Corners found in image ../../images/virtual/set_5/image4.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image1.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image2.jpg
(480, 480)


Corners found in image ../../images/virtual/set_5/image3.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image5.jpg
Corners found in 5 images
(480, 480)
RMS re-projection error: 0.23174696824654417
The median re-projection error 0.22990884012875903
Camera Matrix:
 [[422.14661882   0.         238.57909834]
 [  0.         421.805195   239.85147633]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[ 8.57694379e+01 -3.63430067e+02  3.73762212e-05  4.32801116e-04
  -8.65594325e+02  8.59439982e+01 -3.64239568e+02 -8.66586078e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]

 Using set 10
0
Corners found in image ../../images/virtual/set_10/image10.jpg
(480, 480)
Corners found in image ../../images/virtual/set_10/image4.jpg
(480, 480)
Corners found in image ../../images/virtual/set_10/image8.jpg
(480, 480)
Corners found in image ../../images/virtual/set_10/image1.jpg
(480, 480)
Corners found in image 

In [148]:

# testing using the image set 5

image_set_5 = glob.glob(images_set[0])

cameraMatrix, distCoeff, rvecs, tvecs, stdIntrinc, stdExtrinsic, perViewError = calibrate(image_set_5, useFisheye, patternSize,squareSize, imgSize) 

0
Corners found in image ../../images/virtual/set_5/image4.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image1.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image2.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image3.jpg
(480, 480)
Corners found in image ../../images/virtual/set_5/image5.jpg
Corners found in 5 images
(480, 480)
RMS re-projection error: 0.23174696824654417
The median re-projection error 0.22990884012875903
Camera Matrix:
 [[422.14661882   0.         238.57909834]
 [  0.         421.805195   239.85147633]
 [  0.           0.           1.        ]]
Distortion Parameters:
 [[ 8.57694379e+01 -3.63430067e+02  3.73762212e-05  4.32801116e-04
  -8.65594325e+02  8.59439982e+01 -3.64239568e+02 -8.66586078e+02
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [149]:
rvecs[0]

array([[ 4.35636252e-01],
       [ 1.50076362e-05],
       [-2.25453527e-05]])

In [150]:
image_of_interest = glob.glob(image_set_5[0])
board_points = construct3DPoints(patternSize=(7,7), squareSize=30) 
real_world_points, real_image_points, _ = detectCorners(images=image_of_interest, boardPoints=board_points,patternSize=(7,7))

error = check_reprojection_error(real_image_points[0], real_world_points[0], rvecs[0], tvecs[0], cameraMatrix, distCoeff)

projected_image_points = cv2.projectPoints(real_world_points[0], rvecs[0], tvecs[0], cameraMatrix, distCoeff)


error2 = cv2.norm(real_image_points[0], projected_image_points[0], normType= cv2.NORM_L2) / len(projected_image_points[0])

0
Corners found in image ../../images/virtual/set_5/image4.jpg
Corners found in 1 images
(480, 480)


In [151]:
error2

0.03431017566435014

---

In [152]:
np.median(perViewError)

0.22990884012875903

# Funciona utilizando os dados do OpenCV
img = cv2.imread(images[0])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)

_, rvec, tvec = cv2.solvePnP(boardPoints, corners, cameraMatrix, 0)

print(rvecs[0])
print('\n', rvec)
 

# Testar usando o focal lenght da estimativa do Deep Calib
f = 421.33416748046875
o_x, o_y = (240,240)
camera_matrix = np.array([[  f,   0, o_x],
                          [  0,   f, o_y],
                          [  0,   0,   1]])

_, rvec, tvec = cv2.solvePnP(boardPoints, corners, camera_matrix, 0)

image_points_deep, _ = cv2.projectPoints(boardPoints, rvec, tvec, camera_matrix, 0)

image_points_deep = image_points_deep.reshape(49,2)
corners = corners.reshape(49,2)

error = np.linalg.norm(image_points_deep - corners, axis =1)

# error_median = np.mean(error)
# error_median

O que eu preciso avaliar agora é: 
- E se eu aumentar o tamanho do set de imagens? (Usar mais imagens)
- E se eu usar uma padrão de calibraçao maior? (Usar mais correspondências)
- E se eu usar uma distorção maior (Cameras fisheye)
- E se eu adicionar ruído na imagem? (Detecção dos cornes ja não é mais fiel)

Para a calibração deep:

Para set 5: 0.14613342

Para set 10: 0.15971756

Para set 15: 0.1541214

Para set 20: 0.14318848

Para set 25: 0.15511322 